# Neptune Analytics Instance Management Lifecycle Demo

This notebook demonstrates how to:
1. Create a new Neptune Analytics instance on-demand
2. Configure the NetworkX backend to use the newly created instance
3. Run graph algorithms on the provisioned instance
4. Understand the complete lifecycle of Neptune Analytics resources

The notebook uses the `create_na_instance` function to provision a new Neptune Analytics instance when needed.

## Setup

Import the necessary libraries and set up logging.

In [ ]:
import asyncio
import logging
import sys
import os

import boto3
import networkx as nx
from nx_neptune import NeptuneGraph
from nx_neptune.instance_management import create_na_instance, import_csv_from_s3
import matplotlib.pyplot as plt

In [ ]:
# Configure logging to see detailed information about the instance creation process
logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout  # Explicitly set output to stdout
)
# Enable debug logging for the instance management module
for logger_name in ['nx_neptune.instance_management']:
    logging.getLogger(logger_name).setLevel(logging.DEBUG)
logger = logging.getLogger(__name__)

## Configuration

Check for environment variables and configure the NetworkX backend for Neptune Analytics.

In [ ]:
def check_env_vars(var_names):
    values = {}
    for var_name in var_names:
        value = os.getenv(var_name)
        if not value:
            print(f"Warning: Environment Variable {var_name} is not defined")
            print(f"You can set it using: %env {var_name}=your-value")
        else:
            print(f"Using {var_name}: {value}")
        values[var_name] = value
    return values
    
# Check for optional environment variables
env_vars = check_env_vars(['ARN_S3_IMPORT', 'ARN_IAM_ROLE', 'GRAPH_ID'])

# Get environment variables
s3_location_import = os.getenv('ARN_S3_IMPORT')  # Optional: for importing data after creation
role_arn = os.getenv('ARN_IAM_ROLE')  # Required for S3 import
graph_id = os.getenv('GRAPH_ID')  # Optional: if you want to use an existing instance

## Configure NetworkX Backend

Set up the NetworkX configuration to enable automatic instance creation.

In [ ]:
# Access the NetworkX configuration for the Neptune backend
nx_config = nx.config['backends']['neptune']


## Option 1 - Create a New Neptune Analytics Instance

Provision a new Neptune Analytics instance on demand. This process may take several minutes to complete.

In [ ]:
nx_config['create_instance'] = True
# Create a new Neptune Analytics instance
future = create_na_instance()

# Wait for the instance to be created and available
await future

# Get the graph_id of the newly created instance
new_graph_id = future.result()
print(f"Successfully created a new Neptune Analytics instance with graph_id: {new_graph_id}")

# Update the NetworkX configuration with the new graph_id
nx_config['graph_id'] = new_graph_id

## Option 2 - Create a New Neptune Analytics Instance with s3 data

Provision a new Neptune Analytics instance on demand. This process may take several minutes to complete.

In [ ]:
nx_config['graph_id'] = ''
nx_config['create_instance'] = True
nx_config['s3_import_path'] = s3_location_import
nx_config['role_arn'] = role_arn

# Create a new Neptune Analytics instance
future = create_na_instance()

# Wait for the instance to be created and available
await future

# Get the graph_id of the newly created instance
graph_id = future.result()
print(f"Successfully created a new Neptune Analytics instance wirh air route dataset.")


# Update the NetworkX configuration with the new graph_id
nx_config['graph_id'] = graph_id
os.environ['GRAPH_ID'] = graph_id


## Initialize a Graph and Connect to Neptune Analytics

Create a NetworkX graph and initialize the connection to the Neptune Analytics instance.

In [ ]:
BACKEND = "neptune"
g = nx.DiGraph()
# BFS on Air route
r = list(nx.bfs_edges(g, source="48", backend=BACKEND))
print('BFS search on Neptune Analytics with source=48 (Vancouver international airport): ')
print(f"Total size of the result: {len(r)}")


## Conclusion

This notebook demonstrated the complete lifecycle of a Neptune Analytics instance:

1. **Creation**: We created a new Neptune Analytics instance on demand
2. **Configuration**: We configured NetworkX to use the new instance
3. **Usage**: We ran graph algorithms on the instance
4. **Management**: We discussed options for managing the instance lifecycle

The `create_na_instance()` function makes it easy to provision Neptune Analytics resources when needed, enabling seamless scaling of graph computations without changing your code.